# Train - Alexnet

In [1]:
import gc, datetime
import cv2
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from utilities import load_images, get_multi_hot_labels
from alexnet_model import alexnet_model_fn
tf.logging.set_verbosity(tf.logging.INFO)
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [2]:
train_df = pd.read_csv('data/train_without_missing.csv')
validation_df = pd.read_csv('data/validation.csv')

train_path_list = train_df['imagePath']
eval_path_list = validation_df['imagePath']

eval_data = load_images(eval_path_list)
eval_labels = get_multi_hot_labels(validation_df, list(range(validation_df.shape[0])))

Loading images: 0
Loading images: 1000
Loading images: 2000
Loading images: 3000
Loading images: 4000
Loading images: 5000
Loading images: 6000
Loading images: 7000
Loading images: 8000
Loading images: 9000


In [3]:
eval_data.shape, eval_labels.shape

((9897, 224, 224, 3), (9897, 228))

In [4]:
def main():
    
    train_iter_size = 10000
    num_iters = 150
    batch_size = 1
    eval_every_iters = 1
    np.random.seed(123)
    
    #train_steps = []
    #train_losses = []
    
    eval_steps = []
    eval_losses = []
    eval_precision = []
    eval_recall = []
    eval_meanfscore = []
    
    # Create the Estimator
    multilabel_classifier = tf.estimator.Estimator(
        model_fn=alexnet_model_fn, model_dir="model/multilabel_alexnet_model")

    # Set up logging for predictions
    #tensors_to_log = {"probabilities": "sigmoid_tensor"}
    #tensors_to_log = {"meanfscore": "eval_tensor"}
    tensors_to_log = []
    logging_hook = tf.train.LoggingTensorHook(
        tensors=tensors_to_log, every_n_iter=1000)
    
    for k in range(num_iters):
        print('Trained images so far: {}'.format(k * train_iter_size))
        
        # Randomly load training data and labels
        print('Loading train images..')
        random_indices = np.random.randint(0, train_df.shape[0], size=train_iter_size)        
        train_paths = [train_path_list[i] for i in random_indices]
        train_data = load_images(train_paths)
        
        print('Loading train labels..')
        train_labels = get_multi_hot_labels(train_df, random_indices)

        # Train the model
        train_input_fn = tf.estimator.inputs.numpy_input_fn(
            x={"x": train_data},
            y=train_labels,
            batch_size=batch_size,
            num_epochs=1,
            shuffle=True)
        multilabel_classifier.train(
            input_fn=train_input_fn,
            hooks=[logging_hook])
        
        if k % eval_every_iters == 0:
            # Evaluate the model and print results
            eval_input_fn = tf.estimator.inputs.numpy_input_fn(
                x={"x": eval_data},
                y=eval_labels,
                shuffle=False)
            eval_results = multilabel_classifier.evaluate(input_fn=eval_input_fn)
            print(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
            print(eval_results)
            
            with open('log/loss_alexnet_lrdecay.csv', 'a') as loss_file:
                loss_file.write(str(eval_results['global_step'])+','+str(eval_results['loss'])+'\n')
            with open('log/score_alexnet_lrdecay.csv', 'a') as score_file:
                score_file.write(str(eval_results['global_step'])+','+str(eval_results['meanfscore'])+','+str(eval_results['precision_micro'])+','+str(eval_results['recall_micro'])+'\n')
            
            eval_steps.append(eval_results['global_step'])
            eval_losses.append(eval_results['loss'])
            eval_precision.append(eval_results['precision_micro'])
            eval_recall.append(eval_results['recall_micro'])
            eval_meanfscore.append(eval_results['meanfscore'])
        
        # Garbage collection
        train_data = None
        train_labels = None
        gc.collect()
    
    eval_track = {'eval_steps':eval_steps, 
                  'eval_losses':eval_losses, 
                  'eval_precision':eval_precision, 
                  'eval_recall':eval_recall, 
                  'eval_meanfscore':eval_meanfscore}
    
    return eval_track

In [ ]:
eval_track = main()

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'model/multilabel_alexnet_model', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x0000028100203DA0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
Trained images so far: 0
Loading train images..
Loading images: 0
Loading images: 1000
Loading images: 2000
Loading images: 3000
Loading images: 4000
Loading images: 5000
Loading images: 6000
Loading images: 7000
Loading images: 8000
Loading images: 9000
Loading train labels..
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn

INFO:tensorflow:loss = 0.069617584, step = 6800 (2.591 sec)
INFO:tensorflow:global_step/sec: 38.4149
INFO:tensorflow:loss = 0.07024329, step = 6900 (2.619 sec)
INFO:tensorflow:global_step/sec: 38.831
INFO:tensorflow: (25.918 sec)
INFO:tensorflow:loss = 0.07755178, step = 7000 (2.560 sec)
INFO:tensorflow:global_step/sec: 38.5865
INFO:tensorflow:loss = 0.085621804, step = 7100 (2.592 sec)
INFO:tensorflow:global_step/sec: 38.7124
INFO:tensorflow:loss = 0.1118279, step = 7200 (2.583 sec)
INFO:tensorflow:global_step/sec: 38.5915
INFO:tensorflow:loss = 0.07650591, step = 7300 (2.591 sec)
INFO:tensorflow:global_step/sec: 38.2247
INFO:tensorflow:loss = 0.08570915, step = 7400 (2.616 sec)
INFO:tensorflow:global_step/sec: 38.1363
INFO:tensorflow:loss = 0.08255237, step = 7500 (2.622 sec)
INFO:tensorflow:global_step/sec: 38.5261
INFO:tensorflow:loss = 0.07296524, step = 7600 (2.596 sec)
INFO:tensorflow:global_step/sec: 37.4838
INFO:tensorflow:loss = 0.057907537, step = 7700 (2.669 sec)
INFO:tenso

INFO:tensorflow:global_step/sec: 38.1942
INFO:tensorflow:loss = 0.0934, step = 13300 (2.616 sec)
INFO:tensorflow:global_step/sec: 37.9869
INFO:tensorflow:loss = 0.123279616, step = 13400 (2.632 sec)
INFO:tensorflow:global_step/sec: 36.7848
INFO:tensorflow:loss = 0.06510405, step = 13500 (2.719 sec)
INFO:tensorflow:global_step/sec: 37.6212
INFO:tensorflow:loss = 0.08808499, step = 13600 (2.658 sec)
INFO:tensorflow:global_step/sec: 38.3634
INFO:tensorflow:loss = 0.080234274, step = 13700 (2.607 sec)
INFO:tensorflow:global_step/sec: 37.8576
INFO:tensorflow:loss = 0.124505885, step = 13800 (2.641 sec)
INFO:tensorflow:global_step/sec: 37.431
INFO:tensorflow:loss = 0.1256688, step = 13900 (2.672 sec)
INFO:tensorflow:global_step/sec: 37.4717
INFO:tensorflow: (26.565 sec)
INFO:tensorflow:loss = 0.11333252, step = 14000 (2.669 sec)
INFO:tensorflow:global_step/sec: 38.2971
INFO:tensorflow:loss = 0.09497093, step = 14100 (2.611 sec)
INFO:tensorflow:global_step/sec: 37.8268
INFO:tensorflow:loss = 

INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from model/multilabel_alexnet_model\model.ckpt-20000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 20001 into model/multilabel_alexnet_model\model.ckpt.
INFO:tensorflow:
INFO:tensorflow:loss = 0.07706573, step = 20000
INFO:tensorflow:global_step/sec: 36.1371
INFO:tensorflow:loss = 0.058846112, step = 20100 (2.767 sec)
INFO:tensorflow:global_step/sec: 37.0418
INFO:tensorflow:loss = 0.0723972, step = 20200 (2.700 sec)
INFO:tensorflow:global_step/sec: 36.8584
INFO:tensorflow:loss = 0.09941642, step = 20300 (2.713 sec)
INFO:tensorflow:global_step/sec: 37.9901
INFO:tensorflow:loss = 0.1436765, step = 20400 (2.632 sec)
INFO:tensorflow:global_step/sec: 37.6255
INFO:tensorflow:loss = 0.07727941, step = 20500 (2.658 sec)
INFO:tensorflow:global_step/sec: 38.3193
INFO:tensorflow:loss = 0.026339889, step = 20600 (2.625 sec)
INFO:tensorflow:global_ste

INFO:tensorflow:loss = 0.09229052, step = 27600 (2.678 sec)
INFO:tensorflow:global_step/sec: 38.2967
INFO:tensorflow:loss = 0.12235475, step = 27700 (2.613 sec)
INFO:tensorflow:global_step/sec: 37.6284
INFO:tensorflow:loss = 0.061493445, step = 27800 (2.656 sec)
INFO:tensorflow:global_step/sec: 38.2344
INFO:tensorflow:loss = 0.07185296, step = 27900 (2.615 sec)
INFO:tensorflow:global_step/sec: 37.7468
INFO:tensorflow: (26.390 sec)
INFO:tensorflow:loss = 0.06599729, step = 28000 (2.649 sec)
INFO:tensorflow:global_step/sec: 38.1662
INFO:tensorflow:loss = 0.035865095, step = 28100 (2.620 sec)
INFO:tensorflow:global_step/sec: 37.6241
INFO:tensorflow:loss = 0.09744747, step = 28200 (2.658 sec)
INFO:tensorflow:global_step/sec: 37.5203
INFO:tensorflow:loss = 0.046568915, step = 28300 (2.665 sec)
INFO:tensorflow:global_step/sec: 36.5647
INFO:tensorflow:loss = 0.06222932, step = 28400 (2.735 sec)
INFO:tensorflow:global_step/sec: 38.1554
INFO:tensorflow:loss = 0.0925973, step = 28500 (2.636 sec)

...

INFO:tensorflow:global_step/sec: 36.0641
INFO:tensorflow:loss = 0.045096867, step = 663600 (2.773 sec)
INFO:tensorflow:global_step/sec: 37.659
INFO:tensorflow:loss = 0.06878534, step = 663700 (2.655 sec)
INFO:tensorflow:global_step/sec: 37.5121
INFO:tensorflow:loss = 0.07142129, step = 663800 (2.666 sec)
INFO:tensorflow:global_step/sec: 37.7928
INFO:tensorflow:loss = 0.08163732, step = 663900 (2.646 sec)
INFO:tensorflow:global_step/sec: 36.9317
INFO:tensorflow: (26.817 sec)
INFO:tensorflow:loss = 0.048608292, step = 664000 (2.708 sec)
INFO:tensorflow:global_step/sec: 37.0419
INFO:tensorflow:loss = 0.033197228, step = 664100 (2.700 sec)
INFO:tensorflow:global_step/sec: 36.2951
INFO:tensorflow:loss = 0.093576096, step = 664200 (2.755 sec)
INFO:tensorflow:global_step/sec: 37.928
INFO:tensorflow:loss = 0.03492508, step = 664300 (2.637 sec)
INFO:tensorflow:global_step/sec: 36.0393
INFO:tensorflow:loss = 0.069620416, step = 664400 (2.775 sec)
INFO:tensorflow:global_step/sec: 36.8772
INFO:ten

INFO:tensorflow:Saving checkpoints for 670001 into model/multilabel_alexnet_model\model.ckpt.
INFO:tensorflow:
INFO:tensorflow:loss = 0.06658298, step = 670000
INFO:tensorflow:global_step/sec: 35.8005
INFO:tensorflow:loss = 0.09620617, step = 670100 (2.793 sec)
INFO:tensorflow:global_step/sec: 37.5431
INFO:tensorflow:loss = 0.08073428, step = 670200 (2.664 sec)
INFO:tensorflow:global_step/sec: 35.3159
INFO:tensorflow:loss = 0.07281383, step = 670300 (2.832 sec)
INFO:tensorflow:global_step/sec: 36.9162
INFO:tensorflow:loss = 0.08053993, step = 670400 (2.709 sec)
INFO:tensorflow:global_step/sec: 37.1903
INFO:tensorflow:loss = 0.091059156, step = 670500 (2.689 sec)
INFO:tensorflow:global_step/sec: 36.6815
INFO:tensorflow:loss = 0.03570226, step = 670600 (2.726 sec)
INFO:tensorflow:global_step/sec: 36.9238
INFO:tensorflow:loss = 0.04805185, step = 670700 (2.708 sec)
INFO:tensorflow:global_step/sec: 37.4167
INFO:tensorflow:loss = 0.0627703, step = 670800 (2.673 sec)
INFO:tensorflow:global_s

INFO:tensorflow:global_step/sec: 37.9542
INFO:tensorflow:loss = 0.11401229, step = 677800 (2.635 sec)
INFO:tensorflow:global_step/sec: 36.4175
INFO:tensorflow:loss = 0.0625506, step = 677900 (2.746 sec)
INFO:tensorflow:global_step/sec: 37.2034
INFO:tensorflow: (27.201 sec)
INFO:tensorflow:loss = 0.049952783, step = 678000 (2.688 sec)
INFO:tensorflow:global_step/sec: 36.8915
INFO:tensorflow:loss = 0.046709776, step = 678100 (2.711 sec)
INFO:tensorflow:global_step/sec: 37.122
INFO:tensorflow:loss = 0.07195116, step = 678200 (2.694 sec)
INFO:tensorflow:global_step/sec: 36.9901
INFO:tensorflow:loss = 0.04167283, step = 678300 (2.704 sec)
INFO:tensorflow:global_step/sec: 38.6582
INFO:tensorflow:loss = 0.08549096, step = 678400 (2.586 sec)
INFO:tensorflow:global_step/sec: 36.5039
INFO:tensorflow:loss = 0.07522816, step = 678500 (2.739 sec)
INFO:tensorflow:global_step/sec: 37.5772
INFO:tensorflow:loss = 0.067947164, step = 678600 (2.661 sec)
INFO:tensorflow:global_step/sec: 36.254
INFO:tensor

INFO:tensorflow:loss = 0.083769515, step = 684100 (2.700 sec)
INFO:tensorflow:global_step/sec: 37.3835
INFO:tensorflow:loss = 0.049640976, step = 684200 (2.675 sec)
INFO:tensorflow:global_step/sec: 36.4253
INFO:tensorflow:loss = 0.063385464, step = 684300 (2.745 sec)
INFO:tensorflow:global_step/sec: 36.3793
INFO:tensorflow:loss = 0.062678024, step = 684400 (2.749 sec)
INFO:tensorflow:global_step/sec: 36.7873
INFO:tensorflow:loss = 0.060797926, step = 684500 (2.718 sec)
INFO:tensorflow:global_step/sec: 36.479
INFO:tensorflow:loss = 0.080095425, step = 684600 (2.741 sec)
INFO:tensorflow:global_step/sec: 37.2982
INFO:tensorflow:loss = 0.04443681, step = 684700 (2.681 sec)
INFO:tensorflow:global_step/sec: 37.0776
INFO:tensorflow:loss = 0.047625437, step = 684800 (2.697 sec)
INFO:tensorflow:global_step/sec: 37.6159
INFO:tensorflow:loss = 0.07957214, step = 684900 (2.658 sec)
INFO:tensorflow:global_step/sec: 37.4394
INFO:tensorflow: (27.036 sec)
INFO:tensorflow:loss = 0.07842342, step = 6850

INFO:tensorflow:global_step/sec: 36.8866
INFO:tensorflow:loss = 0.05762161, step = 690500 (2.695 sec)
INFO:tensorflow:global_step/sec: 37.4283
INFO:tensorflow:loss = 0.07297708, step = 690600 (2.672 sec)
INFO:tensorflow:global_step/sec: 37.228
INFO:tensorflow:loss = 0.11545829, step = 690700 (2.686 sec)
INFO:tensorflow:global_step/sec: 36.2878
INFO:tensorflow:loss = 0.04515925, step = 690800 (2.756 sec)
INFO:tensorflow:global_step/sec: 37.339
INFO:tensorflow:loss = 0.016028672, step = 690900 (2.694 sec)
INFO:tensorflow:global_step/sec: 37.1907
INFO:tensorflow: (27.199 sec)
INFO:tensorflow:loss = 0.12686066, step = 691000 (2.689 sec)
INFO:tensorflow:global_step/sec: 36.9838
INFO:tensorflow:loss = 0.070499286, step = 691100 (2.688 sec)
INFO:tensorflow:global_step/sec: 37.1889
INFO:tensorflow:loss = 0.08757684, step = 691200 (2.705 sec)
INFO:tensorflow:global_step/sec: 38.0055
INFO:tensorflow:loss = 0.03502934, step = 691300 (2.616 sec)
INFO:tensorflow:global_step/sec: 36.5004
INFO:tensor

In [ ]:
# eval loss plot
plt.figure(figsize=(12,8))
plt.plot(eval_track['eval_steps'], eval_track['eval_losses'])
plt.xlabel("Step")
plt.ylabel("Validation loss")

In [ ]:
# eval score plot
plt.figure(figsize=(12,8))
plt.plot(eval_track['eval_steps'], eval_track['eval_meanfscore'], label = 'meanfscore')
plt.plot(eval_track['eval_steps'], eval_track['eval_precision'], label = 'precision')
plt.plot(eval_track['eval_steps'], eval_track['eval_recall'], label = 'recall')
plt.legend()
plt.xlabel("Step")
plt.ylabel("Score")